In [ ]:
from pyspark import SparkContext
sc=SparkContext(master="local[*]", appName="graph")


# VERSION 1 / Sans accumulateur

In [119]:
file=sc.textFile('./graph_input.txt')

In [120]:
file.collect()

['1\t2,5|GRIS|0',
 '2\t3,4|BLANC|-1',
 '3\t6|BLANC|-1',
 '4\t|BLANC|-1',
 '5\t6|BLANC|-1',
 '6\t|BLANC|-1']

In [121]:
graph =file.flatMap(lambda x: [x.split("\t")])
graph.collect()

[['1', '2,5|GRIS|0'],
 ['2', '3,4|BLANC|-1'],
 ['3', '6|BLANC|-1'],
 ['4', '|BLANC|-1'],
 ['5', '6|BLANC|-1'],
 ['6', '|BLANC|-1']]

In [122]:
graph1 =graph.flatMap(lambda x: [[[x[0]], x[1].split("|")]])
graph1.collect()

[[['1'], ['2,5', 'GRIS', '0']],
 [['2'], ['3,4', 'BLANC', '-1']],
 [['3'], ['6', 'BLANC', '-1']],
 [['4'], ['', 'BLANC', '-1']],
 [['5'], ['6', 'BLANC', '-1']],
 [['6'], ['', 'BLANC', '-1']]]

In [28]:
#graph2=graph1.map(lambda x: x[1][0].split(","))
#graph2.collect()

[['2', '5'], ['3', '4'], ['6'], [''], ['6'], ['']]

In [155]:
#LANCEMENT 1 

def depth(g):
    l = [] 
    k = g[0]# cle => nom du noeud
    val = g[1] # valeur du noeud  
    #node = g[1][0].split(",")
    nd = g[1][0]# terminaison(s) du noeud 
    for v in g[0]:
        if val[1] == 'GRIS':
            # return  val[0], 'NOIR', int(val[2])
            l.append([ k[0]  , [val[0], 'NOIR', int(val[2])]])
            if nd > "":
                node = g[1][0].split(",")
                for n in node:
                    l.append([n,['', 'GRIS', int(val[2])+1]])
        else: 
            l.append([ k[0], [val[0], val[1], int(val[2])]] )
    return l

graph2=graph1.flatMap(depth)
graph2.collect()

[['1', ['2,5', 'NOIR', 0]],
 ['2', ['', 'GRIS', 1]],
 ['5', ['', 'GRIS', 1]],
 ['2', ['3,4', 'BLANC', -1]],
 ['3', ['6', 'BLANC', -1]],
 ['4', ['', 'BLANC', -1]],
 ['5', ['6', 'BLANC', -1]],
 ['6', ['', 'BLANC', -1]]]

In [156]:
def reducer(a,b):
    if a[2]>b[2]:
        return b[0],'GRIS', a[2]
    else:
        return a[0],'GRIS', b[2]
        
graph3 = graph2.reduceByKey(reducer)
graph3.collect()

[('1', ['2,5', 'NOIR', 0]),
 ('4', ['', 'BLANC', -1]),
 ('2', ('3,4', 'GRIS', 1)),
 ('5', ('6', 'GRIS', 1)),
 ('3', ['6', 'BLANC', -1]),
 ('6', ['', 'BLANC', -1])]

In [157]:
#LANCEMENT 2
graph4=graph3.flatMap(depth)
graph4.collect()

[['1', ['2,5', 'NOIR', 0]],
 ['4', ['', 'BLANC', -1]],
 ['2', ['3,4', 'NOIR', 1]],
 ['3', ['', 'GRIS', 2]],
 ['4', ['', 'GRIS', 2]],
 ['5', ['6', 'NOIR', 1]],
 ['6', ['', 'GRIS', 2]],
 ['3', ['6', 'BLANC', -1]],
 ['6', ['', 'BLANC', -1]]]

In [158]:
graph5 = graph4.reduceByKey(reducer)
graph5.collect()

[('1', ['2,5', 'NOIR', 0]),
 ('4', ('', 'GRIS', 2)),
 ('2', ['3,4', 'NOIR', 1]),
 ('3', ('6', 'GRIS', 2)),
 ('5', ['6', 'NOIR', 1]),
 ('6', ('', 'GRIS', 2))]

In [159]:
#LANCEMENT 3
graph5=graph4.flatMap(depth)
graph5.collect()

[['1', ['2,5', 'NOIR', 0]],
 ['4', ['', 'BLANC', -1]],
 ['2', ['3,4', 'NOIR', 1]],
 ['3', ['', 'NOIR', 2]],
 ['4', ['', 'NOIR', 2]],
 ['5', ['6', 'NOIR', 1]],
 ['6', ['', 'NOIR', 2]],
 ['3', ['6', 'BLANC', -1]],
 ['6', ['', 'BLANC', -1]]]

In [160]:
graph6 = graph5.reduceByKey(reducer)
graph6.collect()

[('1', ['2,5', 'NOIR', 0]),
 ('4', ('', 'GRIS', 2)),
 ('2', ['3,4', 'NOIR', 1]),
 ('3', ('6', 'GRIS', 2)),
 ('5', ['6', 'NOIR', 1]),
 ('6', ('', 'GRIS', 2))]

# VERSION 2 / Avec accumulateur dans mapper

In [167]:
file=sc.textFile('./graph_input.txt')
graph =file.flatMap(lambda x: [x.split("\t")])
graph1 =graph.flatMap(lambda x: [[[x[0]], x[1].split("|")]])

In [193]:
#mapper

# Compteur graphe non noir
nb_no_blck =sc.accumulator(0)

def mapper(g):
    global nb_no_blck
    l = [] 
    k = g[0]# cle => nom du noeud
    val = g[1] # valeur du noeud  
    nd = g[1][0]# terminaison(s) du noeud 
    for v in g[0]:
        if val[1] != 'NOIR':
            nb_no_blck.add(1)
        if val[1] == 'GRIS':
            l.append([ k[0]  , [val[0], 'NOIR', int(val[2])]])
            if nd > "":
                node = g[1][0].split(",")
                for n in node:
                    l.append([n,['', 'GRIS', int(val[2])+1]])
        else: 
            l.append([ k[0], [val[0], val[1], int(val[2])]] )
    return l

In [194]:
#reducer
def reducer(a,b):
    if a[2]>b[2]:
        return b[0],'GRIS', a[2]
    else:
        return a[0],'GRIS', b[2]

In [171]:
graph2=graph1.flatMap(mapper)
graph3 = graph2.reduceByKey(reducer)

In [172]:
graph3.collect()

[('1', ['2,5', 'NOIR', 0]),
 ('4', ['', 'BLANC', -1]),
 ('2', ('3,4', 'GRIS', 1)),
 ('5', ('6', 'GRIS', 1)),
 ('3', ['6', 'BLANC', -1]),
 ('6', ['', 'BLANC', -1])]

In [204]:
nb_no_blck = sc.accumulator(1)

def main():
    file=sc.textFile('./graph_input.txt')
    graph =file.flatMap(lambda x: [x.split("\t")])
    graph1 = graph.flatMap(lambda x: [[[x[0]], x[1].split("|")]])
    #nb_no_blck = sc.accumulator(1)
    tmp = 0
    while nb_no_blck.value > tmp:
        tmp = nb_no_blck.value # variable temporaire
        graph2 = graph1.flatMap(mapper)# transformation
        graph3 = graph2.reduceByKey(reducer) # transformation
        graph3.count()  # action
        graph1 = graph3
        print(f"var temp ==> {tmp}")
        print(f"var nb_no_blck ==> {nb_no_blck.value}")
    return graph3

In [205]:
graph = main()
graph.collect()

var temp ==> 1
var nb_no_blck ==> 7
var temp ==> 7
var nb_no_blck ==> 12
var temp ==> 12
var nb_no_blck ==> 15
var temp ==> 15
var nb_no_blck ==> 16
var temp ==> 16
var nb_no_blck ==> 16


[('1', ['2,5', 'NOIR', 0]),
 ('4', ['', 'NOIR', 2]),
 ('2', ['3,4', 'NOIR', 1]),
 ('3', ['6', 'NOIR', 2]),
 ('6', ['', 'NOIR', 3]),
 ('5', ['6', 'NOIR', 1])]

# VERSION 3 / Avec accumulateur dans reducer

In [207]:
file=sc.textFile('./graph_input.txt')
graph =file.flatMap(lambda x: [x.split("\t")])
graph1 =graph.flatMap(lambda x: [[[x[0]], x[1].split("|")]])

In [208]:
#mapper

# Compteur graphe non noir
nb_no_blck =sc.accumulator(0)

def mapper(g):
    global nb_no_blck
    l = [] 
    k = g[0]# cle => nom du noeud
    val = g[1] # valeur du noeud  
    nd = g[1][0]# terminaison(s) du noeud 
    for v in g[0]:
        if val[1] == 'GRIS':
            l.append([ k[0]  , [val[0], 'NOIR', int(val[2])]])
            if nd > "":
                node = g[1][0].split(",")
                for n in node:
                    l.append([n,['', 'GRIS', int(val[2])+1]])
        else: 
            l.append([ k[0], [val[0], val[1], int(val[2])]] )
    return l

In [209]:
#reducer
def reducer(a,b):
    if a[2]>b[2]:
        nb_no_blck.add(1)
        return b[0],'GRIS', a[2]
    else:
        nb_no_blck.add(1)
        return a[0],'GRIS', b[2]

In [210]:
nb_no_blck = sc.accumulator(1)

def main():
    file=sc.textFile('./graph_input.txt')
    graph =file.flatMap(lambda x: [x.split("\t")])
    graph1 = graph.flatMap(lambda x: [[[x[0]], x[1].split("|")]])
    #nb_no_blck = sc.accumulator(1)
    tmp = 0
    while nb_no_blck.value > tmp:
        tmp = nb_no_blck.value # variable temporaire
        graph2 = graph1.flatMap(mapper)# transformation
        graph3 = graph2.reduceByKey(reducer) # transformation
        graph3.count()  # action
        graph1 = graph3
        print(f"var temp ==> {tmp}")
        print(f"var nb_no_blck ==> {nb_no_blck.value}")
    return graph3

In [211]:
graph = main()
graph.collect()

var temp ==> 1
var nb_no_blck ==> 3
var temp ==> 3
var nb_no_blck ==> 7
var temp ==> 7
var nb_no_blck ==> 9
var temp ==> 9
var nb_no_blck ==> 9


[('1', ['2,5', 'NOIR', 0]),
 ('4', ['', 'NOIR', 2]),
 ('2', ['3,4', 'NOIR', 1]),
 ('3', ['6', 'NOIR', 2]),
 ('6', ['', 'NOIR', 3]),
 ('5', ['6', 'NOIR', 1])]